In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import numpy as np
import time
import random
from pynput.mouse import Button, Controller

In [ ]:
# Initialize Mouse Control
mouse = Controller()
screen_width, screen_height = pyautogui.size()

In [ ]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7, max_num_hands=2)

In [ ]:
# Function to calculate distance between two points
def calculate_distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

In [ ]:
# Function to calculate angle between three points
def calculate_angle(a, b, c):
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    return np.abs(np.degrees(radians))

In [ ]:
# Function to detect a single hand for mouse control
def detect_single_hand(hand_landmarks, frame):
    index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    x, y = int(index_finger_tip.x * screen_width), int(index_finger_tip.y / 2 * screen_height)
    
    # Ensure the mouse position stays within the screen bounds
    x = min(max(x, 0), screen_width - 1)
    y = min(max(y, 0), screen_height - 1)
    pyautogui.moveTo(x, y)

    # Extract landmark positions
    landmark_list = [(lm.x, lm.y) for lm in hand_landmarks.landmark]
    thumb_index_dist = calculate_distance(landmark_list[4], landmark_list[8]) * 1000

    # Left Click
    if thumb_index_dist > 50 and calculate_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50:
        mouse.press(Button.left)
        mouse.release(Button.left)
        cv2.putText(frame, "Left Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        time.sleep(0.2)  # Delay to prevent multiple clicks
    
    # Right Click
    elif thumb_index_dist > 50 and calculate_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50:
        mouse.press(Button.right)
        mouse.release(Button.right)
        cv2.putText(frame, "Right Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        time.sleep(0.2)
    
    # Double Click
    elif thumb_index_dist > 50 and all(calculate_angle(landmark_list[i], landmark_list[i+1], landmark_list[i+3]) < 50 for i in [5, 9]):
        pyautogui.doubleClick()
        cv2.putText(frame, "Double Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        time.sleep(0.2)
    
    # Screenshot
    elif thumb_index_dist < 50 and all(calculate_angle(landmark_list[i], landmark_list[i+1], landmark_list[i+3]) < 50 for i in [5, 9]):
        im1 = pyautogui.screenshot()
        label = random.randint(1, 1000)
        im1.save(f'my_screenshot_{label}.png')
        cv2.putText(frame, "Screenshot Taken", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        time.sleep(0.2)

In [ ]:
# Webcam Capture
cap = cv2.VideoCapture(0)
prev_distance = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    # Detect hand gestures
    if results.multi_hand_landmarks:
        if len(results.multi_hand_landmarks) == 1:
            detect_single_hand(results.multi_hand_landmarks[0], frame)
        
        elif len(results.multi_hand_landmarks) == 2:
            # Zoom functionality with two hands
            hand1 = results.multi_hand_landmarks[0]
            hand2 = results.multi_hand_landmarks[1]

            # Get center points of both hands
            h, w, _ = frame.shape
            hand1_center = (int(hand1.landmark[mp_hands.HandLandmark.WRIST].x * w),
                            int(hand1.landmark[mp_hands.HandLandmark.WRIST].y * h))
            hand2_center = (int(hand2.landmark[mp_hands.HandLandmark.WRIST].x * w),
                            int(hand2.landmark[mp_hands.HandLandmark.WRIST].y * h))

            distance = calculate_distance(hand1_center, hand2_center)

            if prev_distance is not None:
                diff = distance - prev_distance
                if diff > 20:
                    pyautogui.hotkey("ctrl", "+")
                    cv2.putText(frame, "Zoom In", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                elif diff < -20:
                    pyautogui.hotkey("ctrl", "-")
                    cv2.putText(frame, "Zoom Out", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            prev_distance = distance

    cv2.imshow("Hand Gesture Control", frame)
    # Adding a small delay to avoid the frame freeze
    time.sleep(0.01)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()